In [2]:
import os

In [3]:
os.getcwd()

'c:\\Users\\deyar\\OneDrive\\Desktop\\cnnClassifier\\research'

In [4]:
os.chdir('../')

In [5]:
os.getcwd()

'c:\\Users\\deyar\\OneDrive\\Desktop\\cnnClassifier'

In [6]:
from src.cnnClassifier.logger import logging
from src.cnnClassifier.exception import customexception
import sys

In [7]:
from src.cnnClassifier.constants.cont import CONFIG_FILE_PATH,PARAMS_FILE_PATH

In [9]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModel:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_include_top: bool
    params_weights: str



In [10]:
import yaml
from box import ConfigBox

In [14]:
class ConfigManager:
    def __init__(self,paramspath=PARAMS_FILE_PATH,configpath=CONFIG_FILE_PATH):
        try:
         self.config=ConfigBox(yaml.safe_load(open(configpath)))
         self.params=ConfigBox(yaml.safe_load(open(paramspath)))
         os.makedirs(self.config.prepare_base_model.root_dir,exist_ok=True)
         logging.info(f"{self.config.prepare_base_model.root_dir} has been created")
        except Exception as e:
         logging.info("dir is not created")
         raise customexception(e,sys)
    def get_model_config(self):
        param_config=self.params
        config=self.config.prepare_base_model
        base_model_config=PrepareBaseModel(root_dir=config.root_dir,
            base_model_path=config.base_model_path,
            updated_base_model_path=config.updated_base_model_path,
            params_include_top=param_config.INCLUDE_TOP,
            params_weights=param_config.WEIGHTS)
        return base_model_config

In [15]:
base_model_config=ConfigManager()

In [16]:
z=base_model_config.get_model_config()
z

PrepareBaseModel(root_dir='artifacts/prepare_base_model', base_model_path='artifacts/prepare_base_model/base_model.h5', updated_base_model_path='artifacts/prepare_base_model/base_model_updated.h5', params_include_top=False, params_weights='imagenet')

In [17]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Activation,Input,Dense,Conv2D,MaxPool2D,Flatten,Dropout,BatchNormalization,Input,GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

In [18]:
from keras.applications.resnet import ResNet50

In [ ]:
class get_model():
  try:
   def get_base_model(self,b:PrepareBaseModel):
    self.base_model_path=(b.base_model_path)
    self.params_weights=b.params_weights
    self.params_include_top=b.params_include_top
    model=ResNet50(weights=self.params_weights,include_top=self.params_include_top) 
    model.save(self.base_model_path)
    logging.info("base model has been saved")
    return self.base_model_path ,model.summary()
  except Exception as e:
         logging.info("base model has not been saved")
         raise customexception(e,sys)
  def get_updated_model(self,b:PrepareBaseModel):
   try:
    self.updated_base_model_path=b.updated_base_model_path
    self.base_model_path=os.path.join((b.base_model_path))
    self.base_model=load_model(self.base_model_path)
    self.base_model.trainable=False
    model=self.base_model.output
    model= GlobalAveragePooling2D()(model)

    model= Dense(1024)(model)
    model= BatchNormalization()(model)
    model= Activation('relu')(model)
    model = Dropout(0.2)(model)

    model= Dense(128)(model)
    model= BatchNormalization()(model)
    model= Activation('relu')(model)
    model= Dropout(0.1)(model)
    
    output= Dense(4, activation='softmax')(model)
    model = Model(inputs=self.base_model.input, outputs=output)
    model.save(self.updated_base_model_path)
    logging.info("updated base model has been saved")
    model.summary()
    return self.updated_base_model_path
   except Exception as e:
         logging.info("updated base model has not been saved")
         raise customexception(e,sys)
        

In [20]:
basemodel=get_model()

In [21]:
basemodel.get_base_model(z)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 26s 0us/step


Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, None,      │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

('artifacts/prepare_base_model/base_model.h5', None)

In [22]:
basemodel.get_updated_model(z)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, None,      │      1,024 │ conv2_block1_3_c

 Total params: 25,822,212 (98.50 MB)

 Trainable params: 2,232,196 (8.52 MB)

 Non-trainable params: 23,590,016 (89.99 MB)

'artifacts/prepare_base_model/base_model_updated.h5'